In [5]:
#Input Data
#Uncompress the input variables .tar archieve 
#!tar xf var5.tar #Use this instead for five-variables models
!tar xf var3.tar
#!tar xf var2.tar #Use this instead for two-variables models

#Uncompress the Land Use/Land Cover .tar archieve (Contains one file)
!tar xf LULC.tar
#Uncompress the Rainfall  .tar archieve
!tar xf Rainfall.tar


#Output Data
#Uncompress the Soil Moisture  .tar archieve
!tar xf NWM_OUT_SM.tar

In [ ]:
#Runthis
from osgeo import gdal
import os
import numpy as np

#num_of_variables = 5  #Use this instead for five-variables models
num_of_variables = 3    
#num_of_variables = 2  #Use this instead for two-variables models

number_of_entries = (365 + 153)*8 - 2

channels = num_of_variables + 2

#Dimensions of the inut
rows =65 
columns = 55

#Populate a list of input variables files
filesList = sorted(os.listdir('./var'+ str(num_of_variables)))


#filesList = filesList[num_of_variables*4:-(num_of_variables*4)]   #Use this instead for exclusive conv 
filesList = filesList[num_of_variables*5:-(num_of_variables*3)]

#Populate a list of rainfall files
RFFilesList = sorted(os.listdir('./Rainfall'))

#RFFilesList = RFFilesList[5:-3]  #Use this instead for exclusive conv 
RFFilesList = RFFilesList[6:-2]

#Generate a complete list of inputs including Land use / Land Cover
NewCompletelist = []
for i in range(0,len(filesList),num_of_variables):
    for j in range(0,num_of_variables):
        NewCompletelist.append(filesList[i+j])
    NewCompletelist.append(RFFilesList[i//num_of_variables])
    NewCompletelist.append('LULC.tif_1km.tif')

#A function to load input data
def read_input_files(mylist):
    finallist=[]
    count =0
    for fn in (mylist):
        count =count +1
        if 'wrfsfcf' in fn:
            raster = gdal.Open(os.path.join('./var' + str(num_of_variables), fn))
        elif 'GaugeCorr' in fn:
            raster = gdal.Open(os.path.join('./Rainfall', fn))
        else:
            raster = gdal.Open(os.path.join('./LULC', fn))
        if raster is None:
            print ('Unable to open %s')
            break
        band = raster.GetRasterBand(1)
        array = band.ReadAsArray()
        finallist.append(array)   
    return finallist

#Allmerged contains all input data 
Allmerged = np.array(read_input_files(NewCompletelist))

#Allmerged reshaped in the form: (number_of_entries, channels, 65, 55)
length =(Allmerged.shape[0])//(channels )
Allmerged = Allmerged.reshape((length,(channels ),*Allmerged.shape[-2:]))


#Performing Min-Max scalling for the input
for index in range(0, channels):
    max = np.max(Allmerged[:,index,:,:])
    min = np.min(Allmerged[:,index,:,:])
    Allmerged[:,index,:,:] = (Allmerged[:,index,:,:] - min)/(max-min)
    

#Reshaping the input to be in the form: (number_of_entries, 65, 55, channels)
Allmerged= np.moveaxis(Allmerged, 1, -1)
print(Allmerged.shape)


#Filtering the inputs
#By select the inputs corresponding to the exact same hours of the outputs
new_final_input_list =[]   
for i in range(0,Allmerged.shape[0],3): 
    new_final_input_list.append(Allmerged[i])

X = np.array(new_final_input_list)
print(X.shape)

In [ ]:
#A function to load output filenames and data
def read_output_files(foldername):
    finallistnames = []  #A list to store output file names
    finallistdata=[]  #A list to store output data

    path = './' + foldername
    for fn in sorted(os.listdir(path)):
        raster = gdal.Open(os.path.join(path, fn))
        if raster is None:
            print ('Unable to open %s')
            break
        band = raster.GetRasterBand(1)
        array = band.ReadAsArray()
        finallistnames.append(fn)
        finallistdata.append(array)
    return [finallistnames, finallistdata]

SM_output_List = read_output_files('NWM_OUT_SM')

#SMFileNames contains a list of all outputs filenames
SMFileNames = SM_output_List[0][2:]

#SMDataset contains an array of output data 
SMDataset = np.array(SM_output_List[1])[2:]

#Removing negative values
y = SMDataset
y[y<0] =0

#Reshaping the output as (number_of_entries, 65, 55, 1)
y = np.expand_dims(y, axis=3)
y.shape

In [19]:
#Defining the model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense
from keras import callbacks
import numpy as np
import pylab as plt  


model = Sequential()

#First Layer
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=( rows , columns, channels),
                   activation='relu',
                   padding='same', 
                   data_format='channels_last'
                )) 
#batch-norm layer
model.add(BatchNormalization())


#Second Layer
model.add(Conv2D(filters=64, kernel_size=(3, 3),
                   activation='relu',
                   padding='same', 
                   data_format='channels_last'
                )) 
#batch-norm layer
model.add(BatchNormalization())

#Third Layer
model.add(Conv2D(filters=50, kernel_size=(3, 3),
                   activation='relu',
                   padding='same', 
                   data_format='channels_last'
                )) 
#batch-norm layer
model.add(BatchNormalization())

#Fourth Layer
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                   activation='relu',
                   padding='same', 
                   data_format='channels_last'
                )) 
#batch-norm layer
model.add(BatchNormalization())

#Final layer
model.add(Conv2D(filters=1, kernel_size=(1, 1), 
                   activation='sigmoid',
                   padding='same', 
                ))
#Defining loss function,optimizer and model compilation
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [ ]:
#Preparing training, validation and testing datasets

from sklearn.model_selection import train_test_split

#Training+validation dataset
X_reduced = X[0:3038]
y_reduced = y[0:3038]

#splitting into training dataset and  validation dataset
X_train, X_validate, y_train, y_validate = train_test_split(X_reduced, y_reduced,  test_size=0.1, random_state=3)

#Testing dataset
X_test = X[3038:]
y_test = y[3038:]

#Defining
callback = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#Model fitting
history=model.fit(X_train, y_train, batch_size=5, epochs=30,verbose=2, validation_data=(X_validate, y_validate),callbacks=[callback])

In [22]:
#Saving the trained model and its weights
from keras.models import load_model

model.save('Conv_model.h5')  

model.save_weights('Conv_model_weights.h5')

In [24]:
#Generate predictions to the whole dataset (training + validation + test)

predictions = model.predict(X)

#Defining a function to convert a numpy array to raster format

import  osr

def array2raster(newRasterfn,rasterfn,array):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    cols = array.shape[1]
    rows = array.shape[0]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

#create a folder to save the predictions in raster format
!mkdir predicted

for i in range (0,len(SMFileNames)):
  #read original output filename
  originalfilename = SMFileNames[i]

  sample_prediction = np.squeeze(predictions[i]) 

  #Generate predicted output filename
  filename = originalfilename [0:-3] + "predicted"+ ".tif"
  predictedpath = "predicted/"+ filename
  #save theoutput prediction in tif format
  array2raster(predictedpath,'/content/LULC/LULC.tif_1km.tif',sample_prediction)

#Archieve all predictions in one .tar file
!tar cf  predicted_conv.tar predicted